# Annotator assessment

Assses the performances of annotators

In [ ]:
import json
import os
import pandas as pd
import regex

In [ ]:
def compare_str(field, dict_1, dict_2, correct, total):
    if field in total:
        total[field] += 1
    else:
        total[field] = 1
    if dict_1[field] == dict_2[field]:
        if field in correct:
            correct[field] += 1
        else:
            correct[field] = 1
    return correct, total

In [ ]:
def compare_list(field, dict_1, dict_2, correct, total):
    for i in range(0, len(dict_1[field])):
        if type(dict_2[field]) == list and i < len(dict_2[field]):
            for sub_field in dict_1[field][i]:
                if sub_field in dict_2[field][i]:
                    combined_field = f"{field}/{sub_field}"
                    if combined_field in total:
                        total[combined_field] += 1
                    else:
                        total[combined_field] = 1
                    if dict_1[field][i][sub_field] == dict_2[field][i][sub_field]:
                        if combined_field in correct:
                            correct[combined_field] += 1
                        else:
                            correct[combined_field] = 1
    return correct, total

In [ ]:
def analyze(file_name, correct, total):
    data = pd.read_csv(file_name, on_bad_lines='skip')
    skipped = {}
    for index, row in data.iterrows():
        row["answers"] = json.loads(row["answers"])
        row["merged"] = json.loads(row["merged"])
        for annotator in row["answers"]:
            for field in row["merged"]:
                if field in annotator:
                    if type(annotator[field]) == str:
                        correct, total = compare_str(field, annotator, row["merged"], correct, total)
                    elif type(annotator[field]) == list:
                        correct, total = compare_list(field, annotator, row["merged"], correct, total)
                    else:
                        skipped[field] = str(type(annotator[field]))
    print(skipped)
    return correct, total, len(data)

In [ ]:
correct = {}
total = {}
total_count = 0
for file_name in os.listdir("../../data/suriname/"):
    if regex.search("^Project", file_name):
        print(f"processing {file_name}...", end=" ")
        correct, total, count = analyze(f"../../data/suriname/{file_name}", correct, total)
        print(f"({count} rows)")
        total_count += count

In [ ]:
for field in sorted(total.keys()):
    if field in correct:
        print(f"{int(100 * correct[field] / total[field])}% {field}")
    else:
        print(0, field)
for field in sorted(correct.keys()):
    if field not in total:
        print(f"missing field {field} in total!")

In [ ]:
print(total_count)